<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-tools/blob/main/notebooks/eye_ai_image_cropping_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Prerequisites
!pip install git+https://github.com/fair-research/bdbag
!pip install git+https://github.com/informatics-isi-edu/deriva-py
!pip install 'git+https://github.com/informatics-isi-edu/eye-ai-tools' --upgrade --force
!pip install 'git+https://github.com/informatics-isi-edu/deriva-ml' --upgrade --force

import json
import os
from eye_ai import EyeAI
import pandas as pd



  Cloning https://github.com/fair-research/bdbag to /tmp/pip-req-build-3y7z2144
  Running command git clone --filter=blob:none --quiet https://github.com/fair-research/bdbag /tmp/pip-req-build-3y7z2144
  Resolved https://github.com/fair-research/bdbag to commit f8d81362ceaff7a0328dd89257655cfcb3f7d111
  Preparing metadata (setup.py) ... done
  Created wheel for bdbag: filename=bdbag-1.7.2-py2.py3-none-any.whl size=74828 sha256=e396878bb27ff38fb802e4b61e922ddd6bd6e0f2a62e40c0f7975de071e41c7d
  Stored in directory: /tmp/pip-ephem-wheel-cache-amb8lxtj/wheels/e6/37/a1/ed335483cf88440ed32b8f918cc20fa31b2ee2c81dd9504f7a
Successfully built bdbag
  Cloning https://github.com/informatics-isi-edu/deriva-py to /tmp/pip-req-build-xrwa5sd6
  Running command git clone --filter=blob:none --quiet https://github.com/informatics-isi-edu/deriva-py /tmp/pip-req-build-xrwa5sd6
  Resolved https://github.com/informatics-isi-edu/deriva-py to commit 3dda9d8bda15f00434263d3509ef33895fdbdfd5
  Preparing metadata

In [2]:
# @title login to DERIVA via Globus Auth

DEBUG_MODE = True # False #@param
catalog_id = "eye-ai" #@param
DEFAULT_SERVER = 'dev.eye-ai.org' if DEBUG_MODE else 'www.eye-ai.org'

!deriva-globus-auth-utils login --no-browser --host {DEFAULT_SERVER}


2024-01-31 23:22:02,692 - INFO - No default configuration file found, attempting to create one at: /root/.deriva/config.json
2024-01-31 23:22:03,448 - INFO - NoSavedTokens: No tokens were loaded
Please paste the following URL in a browser:
https://auth.globus.org/v2/oauth2/authorize?client_id=8ef15ba9-2b4a-469c-a163-7fd910c9d111&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2Fdev.eye-ai.org%2Fderiva_all+urn%3Aglobus%3Aauth%3Ascope%3Agroups.api.globus.org%3Aview_my_groups_and_memberships+openid&state=_default&response_type=code&code_challenge=tQrE3qk6P8PqMBGHZNsdUBqmfF6aZyKF3t1bz6yge1I&code_challenge_method=S256&access_type=online&prefill_named_grant=Login+from+deriva-client+on+a8637d751dfa+%5BLinux-6.1.58%2B-x86_64-with-glibc2.35%5D+to+hosts+%5Bdev.eye-ai.org%5D+with+requested+scopes+%5Bhttps%3A%2F%2Fauth.globus.org%2Fscopes%2Fdev.eye-ai.org%2Fderiva_all%2C+urn%3Aglobus%3Aauth%3Ascope%3Agroups.api.globus.org%3Aview_my_gro

In [3]:
EA = EyeAI(hostname = DEFAULT_SERVER, catalog_id = catalog_id )

In [5]:
# @title Initiate an Execution
metadata_records, cm = EA.execution_init(metadata_rid="2-5N48")
metadata_records


In [16]:
# @title Clone github repo
for proc in EA.configuration.process:
  !git clone https://github.com/{proc.owner}/{proc.repo}.git

fatal: destination path 'eye-ai-ml' already exists and is not an empty directory.
fatal: destination path 'eye-ai-ml' already exists and is not an empty directory.


In [7]:
# @title Data Preprocessing (Filtering Image.csv for just Field_2 Images)

# Selecting image of angle 2 (Field 2) -- Image angle vocab 2SK6;
bag_path1 = metadata_records['bag_paths'][0]
Dataset_Path = os.path.join(bag_path1, 'data/Image.csv')
Dataset = pd.read_csv(Dataset_Path)

Dataset_Field_2 = Dataset[Dataset['Image_Angle_Vocab'] == "2SK6"]

# Dataset_Field_2.head()
# for sanity check compare the filtered data length with just taking Filename column and filtering it with the same condition where it has Field 2 in it as a substring.

print(len(Dataset_Field_2))
print(len(Dataset[Dataset['Filename'].str.contains("Field 2")]))

file2_csv_path = f"{bag_path1}_Field_2.csv"
Dataset_Field_2.to_csv(file2_csv_path, index=False)

5
5


In [24]:
# @title Execute Proecss algorithm (Cropping)

os.chdir('/content/eye-ai-ml/Glaucoma Suspect or No Glaaucoma/')
with cm as exec:
  !python 'Cleaned_Optic_Disc_Cropping_Algorithm_1_0_3_SVG.py' --bag_path '/content/Dataset_2-5K2C/data/assets/Image/' --csv_path '/content/Dataset_2-5K2C_Field_2.csv' --template_path '/content/template.jpg' --output_directory '../../output/' --model_path '/content/download/Merged_Cropped_Porper_or_Not_Dataset_TLBR_95_VGG19_Val_Accuracy.hdf5' --output_csv_path '../../output/output.csv' --process_rid {metadata_records["process"][0]} --annotation_tag_rid {metadata_records["annotation_tag_rid"]} --annotation_tag_name '{EA.configuration.annotation_tag.name}'

os.chdir('/content')

2024-01-31 05:39:45.200072: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 05:39:45.200139: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 05:39:45.202254: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 05:39:45.214782: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-31 05:39:46.924660: W tensorflow/compiler/tf2

2024-01-31 05:43:06,378 - INFO - Initializing uploader: GenericUploader v1.6.4 [Python 3.10.12, Linux-6.1.58+-x86_64-with-glibc2.35]
2024-01-31 05:43:06,384 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-01-31 05:43:06,387 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-01-31 05:43:06,389 - INFO - Checking for updated configuration...


Exeption type: None, Exeption value: None, Exeption traceback: None


2024-01-31 05:43:07,209 - INFO - Updated configuration found.
2024-01-31 05:43:07,212 - INFO - Attempting to acquire a dependent lock in [/content/ExecutionAssets]
2024-01-31 05:43:07,214 - INFO - Scanning files in directory [/content/ExecutionAssets]...
2024-01-31 05:43:07,218 - INFO - File upload processing completed: 0 files were uploaded successfully, 0 files failed to upload due to errors, 0 files were skipped because they did not satisfy the matching criteria of the configuration.


In [25]:
# @title ML result analysis
import numpy as np
output_csv_path = "output/output.csv"
cropped_info = pd.read_csv(output_csv_path)[["Image RID", "Worked Image Cropping Function"]]

cropped_info['Cropped'] = np.where(cropped_info['Worked Image Cropping Function'] == 'Raw Cropped to Eye', 'False', 'True')
cropped_info = cropped_info[["Image RID", "Cropped"]]
cropped_info.rename(columns={'Image RID': 'RID'}, inplace=True)

In [26]:
# @title Upload ML results
# upload cropping bounding box
EA.upload_assets(f'/content/output/{metadata_records["process"][0]}')

# upload cropping metadata
EA.update_image_table(cropped_info)

2024-01-31 05:43:53,807 - INFO - Initializing uploader: GenericUploader v1.6.4 [Python 3.10.12, Linux-6.1.58+-x86_64-with-glibc2.35]
2024-01-31 05:43:53,810 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-01-31 05:43:53,813 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-01-31 05:43:54,007 - INFO - Checking for updated configuration...
2024-01-31 05:43:54,490 - INFO - Updated configuration found.
2024-01-31 05:43:54,494 - INFO - Scanning files in directory [/content/output/2-5N0Y]...
2024-01-31 05:43:54,502 - INFO - Including file: [/content/output/2-5N0Y/2-5N4E_8GET_imgResize_secondary.svg].
2024-01-31 05:43:54,504 - INFO - Including file: [/content/output/2-5N0Y/2-5N4E_9YSC_imgResize_primary.svg].
2024-01-31 05:43:54,507 - INFO - Including file: [/content/output/2-5N0Y/2-5N4E_9